## Handling NA values of Borough column in New York Accident data for the year 2020

In [2]:
### There are around 25741 records where BOROUGH is na. But for most of them we have latitude and longitude. 
#This notebook does reverse geocoding using geopy.Nominatim with a 1 second delay between requests, and stores the addresses back into the DataFrame
# For large datasets, best practice is to iterate and save results progressively, so we don’t lose progress.

In [3]:
import pandas as pd
import os
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [4]:
# Load the dataset
df = pd.read_csv('F:\\Ganesha_Power_BI\\Ganesha_15\\NYC_Accidents_2020.csv')
df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2020-08-29,15:40:00,BRONX,10466.0,40.89210,-73.833760,POINT (-73.83376 40.8921),PRATT AVENUE,STRANG AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4342908,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,2020-08-29,21:00:00,BROOKLYN,11221.0,40.69050,-73.919914,POINT (-73.919914 40.6905),BUSHWICK AVENUE,PALMETTO STREET,NaN,...,Unspecified,NaN,NaN,NaN,4343555,Sedan,Sedan,NaN,NaN,NaN
2,2020-08-29,18:20:00,NaN,NaN,40.81650,-73.946556,POINT (-73.946556 40.8165),8 AVENUE,NaN,NaN,...,NaN,NaN,NaN,NaN,4343142,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
3,2020-08-29,00:00:00,BRONX,10459.0,40.82472,-73.892960,POINT (-73.89296 40.82472),NaN,NaN,1047 SIMPSON STREET,...,Unspecified,Unspecified,Unspecified,NaN,4343588,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Sedan,Motorcycle,NaN
4,2020-08-29,17:10:00,BROOKLYN,11203.0,40.64989,-73.933890,POINT (-73.93389 40.64989),NaN,NaN,4609 SNYDER AVENUE,...,Unspecified,NaN,NaN,NaN,4342953,Sedan,Sedan,NaN,NaN,NaN


In [5]:
# Check how many are missing values
df['BOROUGH'].isna().sum()

25741

In [6]:
# Specify the columns to check for NA values
columns_to_check = ['BOROUGH', 'LATITUDE', 'LONGITUDE', 'ZIP CODE', 'LOCATION', 'ON STREET NAME', 'OFF STREET NAME']

# Count rows where all of the specified columns have NA values
df[columns_to_check].isna().all(axis=1).sum()

0

In [7]:
OUTPUT_FILE = "reverse_geocoded.csv" # output file with addresses
CHECKPOINT_SIZE = 10                # save every 10 rows

In [8]:
# If output exists, resume from there
if os.path.exists(OUTPUT_FILE):
    df_filtered_borough_nan = pd.read_csv(OUTPUT_FILE, index_col = 0)    
else:
    # Filter rows where 'BOROUGH' has NaN values
    df_filtered_borough_nan = df[df['BOROUGH'].isna()][['LATITUDE', 'LONGITUDE']]

In [9]:
# Add empty address column if not exists
if "address" not in df_filtered_borough_nan.columns:
    df_filtered_borough_nan["address"] = None

In [10]:
df_filtered_borough_nan.head()

,LATITUDE,LONGITUDE,address
2,40.816500,-73.946556,"Food Inc., 2527, Frederick Douglass Boulevard,..."
5,40.682310,-73.844950,"103-19, Woodhaven Boulevard, Ozone Park, Queen..."
7,40.800160,-73.935380,"247, East 121st Street, East Harlem, Manhattan..."
14,40.835373,-73.842186,"1260, Seabury Avenue, Westchester Square, The ..."
15,40.659650,-73.773834,"Queens, Queens County, New York, 11430, United..."


In [11]:
# Initialize geolocator
geolocator = Nominatim(user_agent="my_reverse_geocoder")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

for idx, row in df_filtered_borough_nan.iterrows():
    if pd.notna(row["address"]): 
        continue  # skip already processed
    
    try:
        location = reverse((row["LATITUDE"], row["LONGITUDE"]), language="en")
        df_filtered_borough_nan.at[idx, "address"] = location.address if location else None
    except Exception as e:
        df_filtered_borough_nan.at[idx, "address"] = f"Error: {e}"

    # Save every CHECKPOINT_SIZE rows
    if idx % CHECKPOINT_SIZE == 0:
        df_filtered_borough_nan.to_csv(OUTPUT_FILE)
        print(f"Checkpoint saved at row {idx}")

# Final save
df_filtered_borough_nan.to_csv(OUTPUT_FILE)
print("All done! Results saved to", OUTPUT_FILE)

Checkpoint saved at row 67170
Checkpoint saved at row 67190
Checkpoint saved at row 67280
Checkpoint saved at row 67310
Checkpoint saved at row 67350
Checkpoint saved at row 67370
Checkpoint saved at row 67420
Checkpoint saved at row 67440
Checkpoint saved at row 67450
Checkpoint saved at row 67460
Checkpoint saved at row 67500
Checkpoint saved at row 67510
Checkpoint saved at row 67540
Checkpoint saved at row 67560
Checkpoint saved at row 67570
Checkpoint saved at row 67590
Checkpoint saved at row 67600
Checkpoint saved at row 67630
Checkpoint saved at row 67740
Checkpoint saved at row 67750
Checkpoint saved at row 67770
Checkpoint saved at row 67810
Checkpoint saved at row 67850
Checkpoint saved at row 67860
Checkpoint saved at row 67900
Checkpoint saved at row 67950
Checkpoint saved at row 67960
Checkpoint saved at row 67970
Checkpoint saved at row 67980
Checkpoint saved at row 68020
Checkpoint saved at row 68070
Checkpoint saved at row 68160
Checkpoint saved at row 68180
Checkpoint

In [28]:
# Load the dataset
df_address = pd.read_csv('F:\\Ganesha_Power_BI\\Ganesha_15\\reverse_geocoded.csv', index_col = 0)
df_address.head(25)

,LATITUDE,LONGITUDE,address
2,40.816500,-73.946556,"Food Inc., 2527, Frederick Douglass Boulevard,..."
5,40.682310,-73.844950,"103-19, Woodhaven Boulevard, Ozone Park, Queen..."
7,40.800160,-73.935380,"247, East 121st Street, East Harlem, Manhattan..."
14,40.835373,-73.842186,"1260, Seabury Avenue, Westchester Square, The ..."
15,40.659650,-73.773834,"Queens, Queens County, New York, 11430, United..."
21,40.665840,-73.755510,"Belt Parkway, Laurelton, Queens, Queens County..."
22,40.650520,-73.733090,"256-02, Craft Avenue, Rosedale, Queens, Queens..."
23,40.839680,-73.929276,"Major Deegan Expressway, Morris Heights, The B..."
27,40.674347,-73.820710,"118-02, Sutter Avenue, South Ozone Park, Queen..."
29,40.680954,-73.967680,"Pacific Street, Prospect Heights, Brooklyn, Ki..."


In [30]:
# Remove rows where 'LATITUDE' has NaN values
df_cleaned = df_address.dropna(subset=['LATITUDE'])

In [32]:
df_cleaned.head(25)

,LATITUDE,LONGITUDE,address
2,40.816500,-73.946556,"Food Inc., 2527, Frederick Douglass Boulevard,..."
5,40.682310,-73.844950,"103-19, Woodhaven Boulevard, Ozone Park, Queen..."
7,40.800160,-73.935380,"247, East 121st Street, East Harlem, Manhattan..."
14,40.835373,-73.842186,"1260, Seabury Avenue, Westchester Square, The ..."
15,40.659650,-73.773834,"Queens, Queens County, New York, 11430, United..."
21,40.665840,-73.755510,"Belt Parkway, Laurelton, Queens, Queens County..."
22,40.650520,-73.733090,"256-02, Craft Avenue, Rosedale, Queens, Queens..."
23,40.839680,-73.929276,"Major Deegan Expressway, Morris Heights, The B..."
27,40.674347,-73.820710,"118-02, Sutter Avenue, South Ozone Park, Queen..."
29,40.680954,-73.967680,"Pacific Street, Prospect Heights, Brooklyn, Ki..."


In [38]:
merged_df = pd.merge(df, df_cleaned['address'], left_index=True, right_index=True, how='left')

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,address
0,2020-08-29,15:40:00,BRONX,10466.0,40.892100,-73.833760,POINT (-73.83376 40.8921),PRATT AVENUE,STRANG AVENUE,NaN,...,NaN,NaN,NaN,4342908,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
1,2020-08-29,21:00:00,BROOKLYN,11221.0,40.690500,-73.919914,POINT (-73.919914 40.6905),BUSHWICK AVENUE,PALMETTO STREET,NaN,...,NaN,NaN,NaN,4343555,Sedan,Sedan,NaN,NaN,NaN,NaN
2,2020-08-29,18:20:00,NaN,NaN,40.816500,-73.946556,POINT (-73.946556 40.8165),8 AVENUE,NaN,NaN,...,NaN,NaN,NaN,4343142,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,"Food Inc., 2527, Frederick Douglass Boulevard,..."
3,2020-08-29,00:00:00,BRONX,10459.0,40.824720,-73.892960,POINT (-73.89296 40.82472),NaN,NaN,1047 SIMPSON STREET,...,Unspecified,Unspecified,NaN,4343588,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Sedan,Motorcycle,NaN,NaN
4,2020-08-29,17:10:00,BROOKLYN,11203.0,40.649890,-73.933890,POINT (-73.93389 40.64989),NaN,NaN,4609 SNYDER AVENUE,...,NaN,NaN,NaN,4342953,Sedan,Sedan,NaN,NaN,NaN,NaN
5,2020-08-29,03:29:00,NaN,NaN,40.682310,-73.844950,POINT (-73.84495 40.68231),WOODHAVEN BOULEVARD,NaN,NaN,...,NaN,NaN,NaN,4342721,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,"103-19, Woodhaven Boulevard, Ozone Park, Queen..."
6,2020-08-29,19:30:00,BRONX,10459.0,40.825226,-73.887780,POINT (-73.88778 40.825226),LONGFELLOW AVENUE,EAST 165 STREET,NaN,...,NaN,NaN,NaN,4343004,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN
7,2020-08-29,00:00:00,NaN,NaN,40.800160,-73.935380,POINT (-73.93538 40.80016),2 AVENUE,NaN,NaN,...,NaN,NaN,NaN,4343342,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,"247, East 121st Street, East Harlem, Manhattan..."
8,2020-08-29,19:50:00,BRONX,10466.0,40.894314,-73.860270,POINT (-73.86027 40.894314),EAST 233 STREET,CARPENTER AVENUE,NaN,...,Unspecified,NaN,NaN,4343030,Sedan,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
9,2020-08-29,09:20:00,QUEENS,11385.0,40.706780,-73.908880,POINT (-73.90888 40.70678),NaN,NaN,565 WOODWARD AVENUE,...,NaN,NaN,NaN,4343040,Sedan,NaN,NaN,NaN,NaN,NaN


In [48]:
# Fill NaN values in 'col1' with values from 'col2'
merged_df['BOROUGH'] = merged_df['BOROUGH'].fillna(merged_df['address'].str.split(',').str[-5])
merged_df['ZIP CODE'] = merged_df['ZIP CODE'].fillna(merged_df['address'].str.split(',').str[-2])

In [50]:
merged_df.head(25)

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,address
0,2020-08-29,15:40:00,BRONX,10466.0,40.892100,-73.833760,POINT (-73.83376 40.8921),PRATT AVENUE,STRANG AVENUE,NaN,...,NaN,NaN,NaN,4342908,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
1,2020-08-29,21:00:00,BROOKLYN,11221.0,40.690500,-73.919914,POINT (-73.919914 40.6905),BUSHWICK AVENUE,PALMETTO STREET,NaN,...,NaN,NaN,NaN,4343555,Sedan,Sedan,NaN,NaN,NaN,NaN
2,2020-08-29,18:20:00,Manhattan,10030,40.816500,-73.946556,POINT (-73.946556 40.8165),8 AVENUE,NaN,NaN,...,NaN,NaN,NaN,4343142,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,"Food Inc., 2527, Frederick Douglass Boulevard,..."
3,2020-08-29,00:00:00,BRONX,10459.0,40.824720,-73.892960,POINT (-73.89296 40.82472),NaN,NaN,1047 SIMPSON STREET,...,Unspecified,Unspecified,NaN,4343588,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Sedan,Motorcycle,NaN,NaN
4,2020-08-29,17:10:00,BROOKLYN,11203.0,40.649890,-73.933890,POINT (-73.93389 40.64989),NaN,NaN,4609 SNYDER AVENUE,...,NaN,NaN,NaN,4342953,Sedan,Sedan,NaN,NaN,NaN,NaN
5,2020-08-29,03:29:00,Queens,11417,40.682310,-73.844950,POINT (-73.84495 40.68231),WOODHAVEN BOULEVARD,NaN,NaN,...,NaN,NaN,NaN,4342721,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,"103-19, Woodhaven Boulevard, Ozone Park, Queen..."
6,2020-08-29,19:30:00,BRONX,10459.0,40.825226,-73.887780,POINT (-73.88778 40.825226),LONGFELLOW AVENUE,EAST 165 STREET,NaN,...,NaN,NaN,NaN,4343004,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN
7,2020-08-29,00:00:00,Manhattan,10035,40.800160,-73.935380,POINT (-73.93538 40.80016),2 AVENUE,NaN,NaN,...,NaN,NaN,NaN,4343342,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,"247, East 121st Street, East Harlem, Manhattan..."
8,2020-08-29,19:50:00,BRONX,10466.0,40.894314,-73.860270,POINT (-73.86027 40.894314),EAST 233 STREET,CARPENTER AVENUE,NaN,...,Unspecified,NaN,NaN,4343030,Sedan,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
9,2020-08-29,09:20:00,QUEENS,11385.0,40.706780,-73.908880,POINT (-73.90888 40.70678),NaN,NaN,565 WOODWARD AVENUE,...,NaN,NaN,NaN,4343040,Sedan,NaN,NaN,NaN,NaN,NaN


In [44]:
# Check how many are missing values
merged_df['BOROUGH'].isna().sum()

4563

In [52]:
merged_df.drop(columns=['address'], inplace=True)

In [54]:
merged_df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2020-08-29,15:40:00,BRONX,10466.0,40.89210,-73.833760,POINT (-73.83376 40.8921),PRATT AVENUE,STRANG AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4342908,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,2020-08-29,21:00:00,BROOKLYN,11221.0,40.69050,-73.919914,POINT (-73.919914 40.6905),BUSHWICK AVENUE,PALMETTO STREET,NaN,...,Unspecified,NaN,NaN,NaN,4343555,Sedan,Sedan,NaN,NaN,NaN
2,2020-08-29,18:20:00,Manhattan,10030,40.81650,-73.946556,POINT (-73.946556 40.8165),8 AVENUE,NaN,NaN,...,NaN,NaN,NaN,NaN,4343142,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
3,2020-08-29,00:00:00,BRONX,10459.0,40.82472,-73.892960,POINT (-73.89296 40.82472),NaN,NaN,1047 SIMPSON STREET,...,Unspecified,Unspecified,Unspecified,NaN,4343588,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Sedan,Motorcycle,NaN
4,2020-08-29,17:10:00,BROOKLYN,11203.0,40.64989,-73.933890,POINT (-73.93389 40.64989),NaN,NaN,4609 SNYDER AVENUE,...,Unspecified,NaN,NaN,NaN,4342953,Sedan,Sedan,NaN,NaN,NaN


In [60]:
merged_df.to_csv('cleaned_borough_na.csv', index=False)